In [1]:
import os
import re
from selenium import webdriver
from tqdm import tqdm
from datetime import datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
from selenium.webdriver.chrome.options import Options

In [2]:
#chrome_options = Options()
#chrome_options.add_argument("--headless")
DRIVER_PATH = 'C:/Users/KatjaMeden/Documents/chromedriver_win32/chromedriver.exe'
#driver = webdriver.Chrome(executable_path=DRIVER_PATH, options=chrome_options)
driver = webdriver.Chrome(executable_path=DRIVER_PATH)
wait = WebDriverWait(driver, 10)
driver.get('https://www.dz-rs.si/wps/portal/Home/seje/sejeDT/poDatumu')

C:\Users\KatjaMeden\AppData\Local\Temp\ipykernel_15148\2405478668.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=DRIVER_PATH)


In [3]:
# CLICK CLICK COOKIIIIESSS

cookies = driver.find_element(By.XPATH, "//button[@onclick='acceptCookiees()']")
cookies.click()

In [4]:
# Navigate to VII. mandate data:
menu1 = driver.find_element(By.ID, 'viewns_Z7_J9KAJKG10OK070QT45U8J90GC2_:sf:form1:menu')
menu1.click()

In [5]:
menu2 = driver.find_element(By.XPATH, "//option[@value = 'VIII']")
menu2.click()

In [6]:
menu3 = driver.find_element(By.XPATH, "//input[@value = 'Išči seje']")
menu3.click()

In [7]:
# Create a folder to store the data
folder_name = "Data_test"
os.makedirs(folder_name, exist_ok=True)

progress_bar = tqdm(total=115, desc="Pages Scraped")

while True:
    # Find all links in the table
    links = driver.find_elements(By.XPATH, "//table[@class='dataTableExHov']//a")

    # Loop through the links and extract the href attribute from each element
    index = 0  # Index variable to keep track of the current link
    while index < len(links):
        link = links[index]
        try:
            href = link.get_attribute('href')
            text = link.text
            skupna_seja = None
            if any(keyword in href for keyword in ["seja", "srečanje", "posvet"]):
                if "(skupna seja)" in text:
                    skupna_seja = "(skupna seja)"
                # Click the link
                link.click()

                try:
                    # Check if the "Zapis seje" link exists
                    zapis_seje_links = driver.find_elements(By.XPATH, "//a[contains(@id, 'viewns_Z7_J9KAJKG10OK070QT45U8J90G85_:form1:dataIterator2')]")

                    for i in range(len(zapis_seje_links)):
                        # Find the updated zapis_seje_links elements after the page refresh
                        zapis_seje_links = driver.find_elements(By.XPATH, "//a[contains(@id, 'viewns_Z7_J9KAJKG10OK070QT45U8J90G85_:form1:dataIterator2')]")

                        # Click the i-th "Zapis seje" link
                        zapis_link = zapis_seje_links[i]
                        zapis_link.click()

                        # Wait for the text element to be present
                        wait = WebDriverWait(driver, 10)
                        text_element = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@id ='viewns_Z7_J9KAJKG10G5G80QTKORJHB08M0_:form1:fieldSet1']")))

                        # Get the inner HTML of the text element
                        inner_html = text_element.get_attribute('innerHTML')
                        title_text = driver.find_element(By.XPATH, "//*[@id='viewns_Z7_J9KAJKG10G5G80QTKORJHB08M0_:form1']/table/tbody/tr[1]/td[2]")
                        title = title_text.text
                        date_text = driver.find_element(By.XPATH, "//*[@id='viewns_Z7_J9KAJKG10G5G80QTKORJHB08M0_:form1']/table/tbody/tr[2]/td[2]")
                        date = date_text.text
                        
                        if skupna_seja:
                            title += f" {skupna_seja}"

                         # Generate a unique filename by appending a timestamp with milliseconds
                        timestamp = datetime.now().strftime("%H-%M-%S-%f")[:-3]
                        filename = f"{folder_name}/Seja_{title}_{date}_{timestamp}.html"

                        # Save the inner HTML to a file
                        with open(filename, "w", encoding="utf-8") as file:
                            file.write(inner_html)

                        # Go back to the previous page to continue with other links
                        driver.back()

                except NoSuchElementException:
                    # Go back to the previous page
                    driver.back()

                # Wait for the table to load before finding the links again
                wait = WebDriverWait(driver, 10)
                wait.until(EC.presence_of_element_located((By.XPATH, "//table[@class='dataTableExHov']")))

            # Find all links in the table again
            links = driver.find_elements(By.XPATH, "//table[@class='dataTableExHov']//a")

            # Increment the index to move to the next link
            index += 1

        except (NoSuchElementException, TimeoutException, StaleElementReferenceException):
            # Go back to the previous page
            driver.back()

            # Wait for the table to load before finding the links again
            wait = WebDriverWait(driver, 10)
            wait.until(EC.presence_of_element_located((By.XPATH, "//table[@class='dataTableExHov']")))

            # Find all links in the table again
            links = driver.find_elements(By.XPATH, "//table[@class='dataTableExHov']//a")

            # Increment the index to move to the next link
            index += 1
    try:
        # Find the "Next" button element
        next_button = driver.find_element(By.XPATH, "//input[@id='viewns_Z7_J9KAJKG10OK070QT45U8J90GC2_:sf:form1:tableEx1:deluxe1__pagerNext']")
        next_button.click()

        # Wait for the next page to load (adjust the time as needed)
        driver.implicitly_wait(5)

    except NoSuchElementException:
        # "Next" button is not found, exit the loop
        break

Pages Scraped:   0%|                                                                           | 0/115 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
driver.quit()